# Deep COVID: An Exposition

Throughout our report, we present the contents of 'Deep-COVID: Predicting COVID-19 from chest X-ray images using deep transfer learning' (Deep-COVID, for short) by Minaee et al. We start with an introduction to the research question and overview of the data. Following this, we fit a logistic regression to the classification task. This will be used as our baseline model to compare the models used in the paper against. It also serves as an introduction to the basic concepts of Artificial Neural Networks (ANNs). Using this as a segue, we, next, discuss the basic concepts involved in designing, and training ANN's. Once this is done, we introduce the concepts particular to Convolutional Neural Networks (CNNs). Then, we briefly discuss transfer learning and present the models used in the paper. Finally, we review their model evaluation metrics in comparison with eachother, our initial regression model, current biological-based tests, and discuss potential future work.

## The Goal of Deep-COVID and the Data
### The Goal

The COVID pandemic represents one of the biggest challenges currently facing the United States and the rest of the world. Before vaccination is sufficiently deployed and adopted, efforts towards treatment and prevention are of much importance. One of the most effective tools we have at our disposal is accurate and speedy detection. Diagnosing cases from radiography and radiology image is, perhaps, one of the fastest forms of detection (Minaee et al., 2020). In their paper, Minaee et al. apply transfer learning to fit famous ANNs, including ResNet18, ResNet50, SqueezeNet, and DenseNet-121, to identify COVID-19 disease in analyzed chest X-ray images. Results were promising and it is the hope that these methods can be used to more accurately and quickly identify cases in exposed individuals. We'll discuss, in more detail, the concepts of transfer learning, as well as the basic concepts necessary to understand how the ANNs used are structured. First, however, let's become more acquainted with the data used for training and testing.

### The Data

The authors of Deep-COVID were nice enough to share a subset of their data along with a description of how it was collected, labeled, and organized in a Github repository (https://github.com/shervinmin/DeepCovid/tree/master/data).  Still, we will give a brief synopsis in this section.

The data used by the authors comes from two sources: 
* "Covid-Chestxray-Dataset", for COVID-positive samples
* "ChexPert Dataset", for COVID-negative samples.

<img src = "Sample_images_visualization.jpg" alt = "Visualizations of Data" style = "width : 300px;"/>

From the Covid-Chestxray-Dataset, only images that were selected to have a clear sign of COVID by the authors' board-certified radiologists were used: a total of 71 images, split 31 for training and 40 for testing. Several data-augmentation techniques (as well as over-sampling) were used to increase the number of COVID-19 samples, leading to final training/test split counts of:

<figure>
    <img src = "Data_train_test_counts_table.jpg" alt = "Data_train_test_counts_table" style = "width : 300px;"/>
    <figcaption style = "text-align : center;"> Data train/test split summary (Minaee et al., 2020) </figcaption>
</figure>

We note that one of the limitations of this project is the absence of a larger quantity of data, especially verified COVID-positive images. With the help of more data, we could hope for even better results. Now let's turn to our first model, a logistic regression model.

## Logistic Regression: A Baseline Model and Segue into ANNs

### Introduction and Defining the Model

In this section, we will fit a logitstic regression classifier to a subset of the author's data. This will serve two purposes. On the one hand, it will provide a baseline model to compare against the models used in Deep-COVID. And, on the other, it will serve as a segue into ANNs. Let us briefly recall the functional form of a logistic regression model. Namely, given a binary classification problem (say the two classes are 0 or 1) and a data point $\mathbf{x}$, the probability that $\mathbf{x}$ belongs to class 1 is given by:

$$P(y_i = 1 | \mathbf{x}) = \frac{1}{1 + e ^ {-\mathbf{w}^T \mathbf{x}}}.$$

This functional form can be thought of as the composition of two component functional forms. Namely, a linear component, $\mathbf{z}_i = \mathbf{w} ^ T \mathbf{x} = w_1x_1 + ... + w_px_p$ and a non-linear component, $a_i = \sigma(\mathbf{a}_i)$, where $y_i = a_i$ and $\sigma(x) = \frac{1}{1 + e ^ {-x}}$ denotes a sigmoid function, which we will refer to as a sigmoid *activation* function. Thus, graphically, we can think of our model as follows:

<figure>
    <img src = "Single_Perceptron.png" alt = "Single Perceptron" style = "width : 300px" />
    <figcaption style = "text-align : center;"> Perceptron architecture (Deshpande 2020) </figcaption>
</figure>

In order to train a model of this form to our classification task, we need to define how our input layer will look, i.e. what are our features. In the case of image classification tasks, it is common to use a matrix (2 dimensional for grey-scale images and 3 dimensional for color images) of pixel values of the images as the features. We can then take this matrix and "flatten" it, meaning that we convert it into a 1-Dimensional array (the form necessary for our intput layer above. Furthermore, we assign to each entry of the resulting vector, i.e. each pixel value a weight, and include it in the calculation of our linear component ("Sum" in the above graph). This means our model is "fully connected". Now that we've defined the functional form of our model and it's architecture, let's discuss how we will train it using our training data set. 

### Gradient Based Learning

Most machine learning algorithms, including deep learning algorithms, involve minimizing an objective function, $f(x)$, i.e. finding $x ^ * = \textrm{argmin} f(x)$. If $f(x)$ is differentiable, one common optimization method involves iteratively taking steps in the feature space such that, in each step, we move in a direction likely to furthest reduce $f(x)$. In other words, we want to find the direction, $\mathbf{u}$ such that the directional derivative $D_\mathbf{u} f(x)$ is minimized. (This is an intermediate optimization problem we solve to solve the primary optimization problem of minimizing $f$.) Then, we take a step of a specifized size, known as the "learning rate" (which we'll denote by $\epsilon$) in this direction and repeat.

In order to solve this intermediate optimization propblem, recall how partial derivatives are computed. Namely, given a direction $\mathbf{u}$, the directional derivative of $f(x)$ in the direction of $\mathbf{u}$ at a point, $\mathbf{x_0}$, is given by:

$$D_\mathbf{u} f(\mathbf{x_0}) = \mathbf{u} ^ T \nabla f(\mathbf{x_0})$$

where $\nabla f(\mathbf{x_0})$ is the gradient of $f$ at $\mathbf{x_0}$.

For the purposes of our intermediate optimization problem, we can assume (without loss of generality) that $u$ is a unit vector. Thus, $D_\mathbf{u} f(\mathbf{x_0})$, can be interpreted as the signed length of the projection of $\nabla f(\mathbf{x_0})$ onto the span of $\mathbf{u}$. Using this interpretation, we can now see that the solution to our intermediate problem is given by $-\nabla f(\mathbf{x_0})$. Using this, at each step in our optimization algorithm, we update our choice of $\mathbf{x}$ as follows:

$$\mathbf{x ^ \prime} = \mathbf{x} - \epsilon \nabla f(\mathbf{x}).$$

This process is known as "gradient descent", as we are descending down the objective function according to the gradient. While this constitutes the basis for many popular optimization algorithms used in deep learning, it is (by itself) often not the preferred method. In the next section, we will explain, briefly, some of the defects of the algorithm as well as popular alternatives or fixes.

### Improving Gradient Descent

Gradient descent uses the gradient of $f(x)$ to determine a step direction. In other words, it uses the first derivatives of $f(x)$, making it a "first-order" optimization method. This presents potential problems, including making choosing an appropriate learning rate very difficult, and an innability to effectively deal with saddle points. To illustrate the former, consider the one dimensional case, i.e. $x \in \mathbb{R}$ and $f(x) \in \mathbb{R}$. According to gradient descent, we move with a fixed learning rate in the direction of $f ^ {\prime}(x)$ which is smallest (i.e. negative). If, as we are approaching a local minima, $f ^ {\prime \prime} (x)$ is very large, we run the risk of over shooting the minimum with a larger learning rate, while smaller step sizes can cause the algorithm to be inefficient and take many steps to converge.

<figure>
    <img src = "one_dimensional_GD_step_sizes.png" alt = "first order step size issue" style = "width : 300px" />
    <figcaption style = "text-align : center;"> Illustration of poor step sizes in one dimensional gradient descent (Varma &amp; Das, 2018) </figcaption>
</figure>

This defect persists, analogously, in multiple dimensions. In the image below, we view the trajectory of gradient descent in multiple dimensions using a contour plot representation. Note that the curvature of the objective function is largest in the $[1 \textrm{, } 1] ^ T$ direction and smallest in the $[1 \textrm{, } -1] ^ T$ direction. The objectice function resembles a long canyon (Goodfellow, Bengio & Courville, 2017). Due to a step size which is too large, gradient descent overshoots the bottom of the canyon, having to then descend the opposite canyon wall in the next iteration.

<figure>
    <img src = "inefficient_GD_multivariate.jpg" alt = "multivariate step size defect" style = "width : 200px;"> 
    <figcaption style = "text-align : center;"> A view of gradient descent on an objective function with ill-conditioned Hessian Matrix (Goodfellow, Bengio, & Courville, 2017) </figcaption>
</figure>

One approach to addressing these defects is to use information from the second derivative, or Hessian Matrix (the multivariate equivalent) to modify our step direction, making our approach "second-order". Namely, Newton's Method accomlishes this and can sometimes lead to a critical point much faster than gradient descent. This is not, however, the approach we focus on here.

Another approach is to smooth out the trajectory of gradient descent by using exponentially weighted averages of gradients, $V_{dW}$:

\begin{align*}
    V_{dW} &= 0 &&\textrm{ as an initial value.} \\
    V_{dW} &= \beta V_{dW} + (1 - \beta)dW &&\textrm{ for every successive iteration.}
\end{align*}

And model parameters are updaed by $\theta = \theta - V_{dW}$. Here $\beta$ is a parameter of the algorithm to be set. Intuitively, we can think of this algorithm of taking an approximate exponentially weighted average of the previous $\frac{1}{1 - \beta}$ terms. While this formulation is intuitive to understand, often, in practice, the update expression is written as $V_{dW} = \alpha V_{dW} - \epsilon dW$ and then parameters are updated by $\theta = \theta + V_{dW}$ (where $\epsilon$ need not equal $(1 + \alpha)$). This approach is known as gradient descent with "momentum". 

Yet another approach, known as Root-Mean-Squared-Propogation (RMSprop), modifies the learning rate in each direction differently and inversely proportional to the square root of an exponentially weighted average of the appropriate directional derivative. Namely:

\begin{align*}
    S_{dW} &= \beta S_{dW} + (1 - \beta)dW ^ 2 \qquad \textrm{ where } dW \textrm{ is squared elementwise.}
\end{align*}

And model parameters are updated by:

\begin{align*}
    \theta = \theta - \frac{\epsilon}{\sqrt{S_{dW}} + \delta} dW 
\end{align*}

where division and multiplication are performed element-wise and $\delta$ is a small constant to stabilize the division operation.

One final approach we discuss is known as ADAM optimization. ADAM optimization combines the momentum and RMSprop algorithms together. At each iterative step, we compute $V_{dW}$ as in gradient descent with momentum (except with a bias correction) and $S_{dW}$ as in RMSprop (except with a bias correction, also). Then updates to model parameters occur as:

\begin{align*}
    \theta = \theta -\gamma \frac{V_{dW}}{\sqrt{S_{dW}} + \delta} \textrm{ for learning rate } \gamma
\end{align*}

This is the algorithm the authors to train their models, and the algorithm we use to train our logistic regression.

### Training and Testing A Logistic Regression Model

In the case of binary classification, it is common to use the binary cross-entropy loss function as $f(x)$. Namely, we want to minimize:

$$f(x) = -\frac{1}{N}\sum_{i = 1} ^ N y_i \cdot \textrm{log}(p(y_i)) + (1 - y_i) \cdot \textrm{log}(1 - p(y_i))$$

To implement any of the first order algorithms discussed earlier, we need to determine how to compute the gradient of our loss function with respect to the model parameters, $\nabla_{\theta} J(\theta)$. When information flows forward from the input layer through the model, this is called *forward propogation*. The way we compute $\nabla_{\theta}$ is by sending information from the cost back through the network in a process called *back-propagation*. This is as opposed to computing the analytical expression for $\nabla_{\theta}J(\theta)$ since evaluating this is often computationally expensive as our models get more sophisticated. For the sake of brevity, we will not present the details of back-prop in this report. Instead, we now turn to applying these training methods to our logistic regression model. Namely, we will apply ADAM optimization to our model using the subset of the authors' training data that they provide. The model yields the following evaluation metrics on our test set:

\begin{align*}
    \textrm{Accuracy } \quad \textrm{: } \quad 0.9332 \\
    \textrm{Sensitivity } \quad \textrm{: } \quad 0.7100 \\
    \textrm{Specificity } \quad \textrm{: } \quad 0.9407 \\
    \textrm{AUC } \quad \textrm{: } \quad 0.7665
\end{align*}

As more and more data becomes available to a classification task, the usefullness of deep networks tend to become more and more apparent, as they become more accurate than shallow models (such as our formulation of logistic regression). One such reason for this, is that our logisitc regression model is a linear classifier. Namely, it classifies according to a linear decision boundary. Often, if we suspect a non-linear boundary may be more desirable or realistic, we can account for this limitation by applying the linear classifier not to our data directly ($\mathbf{x}$) but to a non-linear transformation of our data ($\phi(\mathbf{x})$). Instead of having to manually engineer a suitable transformation, the approach of deep learning allows the model to learn $\phi$. In the next section, we'll briefly introduce ANNs and deep learning as a generalization of sorts of the logistic regression model we've developed.

## Introduction to ANNs and Deep Learning

### Introduction

Recall from the previous section that Artificial Neural Networks take on the task of learning a suitable non-linear transformation, $\phi$ to apply to our data. In this spirit, we can think of our network as a composition of functions, $f ^ {(n)}(\phi(\mathbf{x}))$. The reason we attach the $(n)$ superscript is because, in a deep neural network, we model $\phi$ itself as a composition of functions: $f ^ {(n-1)}(f ^ {(n-2)}(...f ^ {(1)}(\mathbf{x})...))$. Thus, in its entirety, a deep neural network can be thought of as a composition of functions: $f ^ {(n)}(f ^ {(n-1)}(...f ^ {(1)}(\mathbf{x})...))$. Using this framework, $f ^ {(1)}$ is considered the *first layer* of the network, $f ^ {(2)}$ is considered the *second layer*, etc. and the length of this chain gives the *depth* of the network. The first layer is often referred to as the *input layer* and the last layer is often referred to as the *output layer*. Layers in between, are often referred to as *hidden layers*. Graphically, we can view these networks as a stacking and composition of multiple logistic regression units (although the sigmoid activation function is often swapped for another -- more on this in a moment):

<figure>
    <img src = "multilayer_perceptron_graph.png" alt = "MLP graph" style = "width : 400px;">
    <figcaption style = "text-align : center;"> MLP graph from (Ogail, 2010). </figcaption>
</figure>

In the figure above, each circle is referred to as a *unit*. Each unit's output value is determined by its inputs as a chosen activation function applied to a linear function of the unit's inputs. The sigmoid function is rarely chosen for input layers. The hyperbolic tangent function is sometimes used, however, the default recommendation for most modern neural networks is the *rectified linear unit* or ReLU function defined by $g(z) = \max\{0, z\}$. Note, we still apply a sigmoid activation at the output layer. More than a slight improvement on the linear classifier of the previous section, the *universal approximation theorems* for neural networks state that a feedforward network with a linear output layer and at least one hidden layer with any appropriate activation functions (sigmoid or ReLU, for example) can approximate any suitably "nice" function from one ﬁnite-dimensional space to another with any desired non-zero amount of error, provided that the network is given enough hidden units or layers. ("nice" can mean slightly different things depending on the specific theorem referenced.)

How are deep neural networks trained? The training methods from the previous discussion of our logistic regression model apply equally well to training deep networks. Namely, gradient based methods are popular and utilize back-prop as a means of computing the gradients. Again, for brevity, we omit the details, but the reader can imagine how the methods of the last section generalize, or really, apply directly to deeper networks. 

Within the context of computer vision tasks, like the one explored in Minaee et al., there is a popular amendment to the basic fully connected networks we've described till now that has proved useful. This amendment constitutes the inclusion of other types of hidden layers, including what're referred to as convolution layers. Fittingly, these models are referred to as Convolutional Neural Networks, or CNNs. These are the class of models Minaee et al. pull from to use in their paper. In the next section, we introduce some of the basic concepts involved with CNNs.

## Introduction to CNNs

Intuitively, we might think that a computer vision task might be made simpler, if we first focus on identifying low level features, like edges, and gradually work up in complexity to things like signs of COVID. While not necessarily a completely accurate description, this conception is a decent heuristic device for thinking about basic convolutional networks. For example, how might we try to force a network to learn about vertical edge detection? One method is to apply a suitable convolution (or sometimes called a filter) to our input arrays of pixels. The *convolution* method of applying such a suitable filter is pictured below:

<figure>
    <img src = "vertical_edge_detection_filter.png" alt = "vertical edge detection" style = "width : 300px;">
    <figcaption style = "text-align : center;"> Vertical edge detection through convolution. (Datahacker.rs., 2019) </figcaption>
</figure>

We see that, for a grey-scale image, the convolution operation --in the CNN sense only; in other areas convolution operations possess different meanings-- consists of overlaying the filter on top of the array, starting at the upper, left hand corner, multiplying the filter values with the pixel values, summing the products together and using the resultant value as the first entry of the output array. We continue in this way, sliding the filter from left to right and top to bottom. It follows, then, that given an input array of size $(n \times n)$ and a filter of size $(f \times f)$, the array resulting from convolution will be of size $(n - f + 1) \times (n - f + 1)$. In practice, instead of hand-picking the filters to use, we usually set the size of the filter and let our optimization algorithm choose the values.

Now, two potential downsides to this procedure include: 1) Every application shrinks the size of your image. And 2) The pixels in the corners of an image and on the edges contribute less to the result than the pixels towards the inner of the image (since they are utilized less often). To address these, we often *pad* the image before convolving. Paddings simply involves adding a number of layers of zeros around the array of pixels. So, if we pad a $(n \times n)$ array with a padding amount, $p$, the resulting array will be of dimensions: $(n + 2p) \times (n + 2p)$. Thus, after applying a filter of size, $(f \times f)$, we're left with an array of size, $(n + 2p - f + 1) \times (n + 2p - f + 1)$. There are two padding practices most commonly used. Namely, 1) no padding. And, 2) Padding such that the resulting array after convolution has the same dimensions as the input array ($p = \frac{f - 1}{2}$, $f$ is usually odd). These are known as *Valid* and *Same* convolutions, respectively.

Another common modification to the convolution layer is to skip a set number, $s$, of columns horizontally and rows vertically every time we shift the filter. This is called a *strided* convolution with stride $s$ and results in an array of size $(\lfloor\frac{n + 2p -f}{s}\rfloor + 1) \times (\lfloor \frac{n + 2p -f}{s}\rfloor + 1)$. Strides, used in conjunction with padding, allow you to lower the size of the output, while not systematically ignoring the borders of the image.

Now, how does this process work in the case of a 3-channel image input as opposed to a grey-scale image input? Instead of convolving with a 2-dimensional array, we convolve with a three dimensional array. Namely, each channel of the filter is convolved with the corresponding channel of the input and the results of those three convolutions are summed to produce the result, a 2-dimensional array. To apply multiple filters at a time (suppose we want to detect vertical and horizontal edges in the same layer), we can apply each of the 3-dimensional filters to our input and each 2-dimensional intermediate result will constitute a single channel of the final 3-dimensional result:

<figure>
    <img src = "3D_convolution_with_3D_output.png" alt = "3D Convolution" style = "width : 400px;">
    <figcaption style = "text-align : center;"> A 3-dimensional convolution with 3-dimensional output. (Datahackers.rs., 2020) </figcaption>
</figure>

There is one more aspect usually found in CNNs that we need to introduce and that is the addition of *pooling* layers. Given a filter and stride size for our pooling layer, for a grey-scale image, we consider only the pixels in the upper, left-hand corner of the image of size ($f \times f$). In "Max-Pooling", the first entry of our result will be determined by the maximum of the pixels being considered. Then, the pixels to be considered moves across and down the image, as with convolutions. Another form of pooling occasionally used is "Average-Pooling" where, instead of taking the maximum of the pixels in consideration, we take the average.

<figure>
    <img src = "pooling_layers.jfif" alt = "Pooling Layers" style = "width : 225px;">
</figure>

The models used by Minaee et al. in Deep-COVID are special network architectures that use these methods. In the next secion, we will introduce the specific models used.

## Models Utilized in Minaee Et al.

## Evaluation Metrics and Future Work

## Bibliography

1) Minaee, Shervin et al. “Deep-COVID: Predicting COVID-19 from chest X-ray images using deep transfer learning.” Medical image analysis vol. 65 (2020): 101794. doi:10.1016/j.media.2020.101794

2) Goodfellow, I., Bengio, Y., &amp; Courville, A. (2017). Deep learning. Cambridge, MA: MIT Press.

3) Varma, S., &amp; Das, S. (2018, September 27). Deep Learning. Retrieved November 24, 2020, from https://srdas.github.io/DLBook/

4) Deshpande, M. (2020, September 29). Perceptrons: The First Neural Networks. Retrieved November 23, 2020, from https://pythonmachinelearning.pro/perceptrons-the-first-neural-networks/

5) Ogail, A. (2010, December 27). Multilayer Perceptron. Retrieved November 25, 2020, from https://elogeel.wordpress.com/2010/05/10/multilayer-perceptron-2/

6) Datahacker.rs. (2019, October 20). #003 CNN More On Edge Detection. Retrieved November 25, 2020, from http://datahacker.rs/edge-detection-extended/

7) Datahacker.rs. (2020, April 28). #006 CNN Convolution On RGB Images. Retrieved November 25, 2020, from http://datahacker.rs/convolution-rgb-image/